# Importance weighted autoencoder in Tensorflow

In [1]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tf.__version__

'1.7.0'

In [4]:
mnist = tf.contrib.learn.datasets.load_dataset('mnist')

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-u

In [8]:
mnist.train.images.shape

(55000, 784)

In [9]:
def xavier_init(fan_in, fan_out, constant=1):
    low = -constant*np.sqrt(6.0/(fan_in + fan_out))
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out),
                            minval=low, maxval=high,
                            dtype=tf.float32)
    

In [ ]:
class IMportanceVariationalAutoencoder(object):
    def __init__(self, network_architechture, sampling_number, transfer_fct=tf.nn.sortplus,
                learning_rate=0.0001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        self.x = tf.placeholder(tf.float32, [None, network_architecture("n_input")])
        self.z_list = []
        self.x_reconstr_mean_list = []
        self.layer_1_list = []
        self.layer_2_list = []
        self.eps_list = []
        self.sampling_number = sampling_number
        
        self.random_call()
        self._create_network()
        self._create_loss_optimizer()
        init = tf.initialize_all_variables()
        
        self.sess = tf.INteractiveSession()
        self.sess.run(init)
        
    def rendom_call(self):
        n_z = self.network_architecture["n_z"]
        epa = tf.random_normal((self.batch_size, n_z), 0, 1, dtype=tf.float32)
        return epa
    
    def _create_network(self):
        network_weights = self._initialize_weights(**self.network_architecture)
        
        self.z_mean, self.z_log_sigma_sq = \
            self._recognitioni_network(network_weights["weights_recog"],
                                       network_weights["biases_recog"])
            
        for i in range(self.sampling_number):
            self.eps_list.append(self.random_call())
            self.z_list.append(tf.add(self.z_mean,
                                     tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq)), self.eps_list[i])))
        
        for i in range(self.sampling_number):
            self.x_reconstr_mean_list.append(self._generator_network(network_weights["weights_gener"],
                                                                    network_weights["biases_gener"], i))
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2,
                           n_hidden_gener_1, n_hidden_gener_2,
                           n_input, n_z)